In [2]:
from pyspark.sql import SparkSession

In [1]:
from pyspark.sql.functions import hour, count, col, expr

In [3]:
spark = SparkSession.builder.appName('Spark_Queries').getOrCreate()

21/07/31 18:27:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.options(inferSchema='True', header='True').csv("retail_data.csv")

In [5]:
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [6]:
df.createTempView("orders")

In [7]:
spark.sql('SELECT HOUR(InvoiceDate) as Hour, COUNT(*) as OrderCount FROM orders GROUP BY HOUR(InvoiceDate)').show()

+----+----------+
|Hour|OrderCount|
+----+----------+
|  12|       421|
|  13|       268|
|  16|       160|
|  15|       189|
|   9|       151|
|  17|       647|
|   8|        46|
|  10|       121|
|  11|       370|
|  14|       735|
+----+----------+



In [8]:
spark.sql('SELECT Country, StockCode, COUNT(StockCode) as ProductBoughtFrequency FROM orders GROUP BY Country, StockCode').show()

+--------------+---------+----------------------+
|       Country|StockCode|ProductBoughtFrequency|
+--------------+---------+----------------------+
|United Kingdom|    21125|                     1|
|United Kingdom|    21794|                     1|
|United Kingdom|    21662|                     2|
|          EIRE|   85136C|                     1|
|United Kingdom|   84032A|                     3|
|United Kingdom|   84086C|                     2|
|United Kingdom|   90214H|                     2|
|United Kingdom|    22753|                     1|
|United Kingdom|    21557|                     2|
|United Kingdom|    21743|                     4|
|United Kingdom|    22488|                     1|
|United Kingdom|    22956|                     3|
|        Norway|    22531|                     1|
|United Kingdom|    22728|                     3|
|United Kingdom|    82551|                     2|
|United Kingdom|    22581|                     2|
|United Kingdom|    21671|                     1|


In [9]:
df.groupBy(hour("InvoiceDate").alias("Hour")).agg(count("*").alias("OrderCount")).show(30)

+----+----------+
|Hour|OrderCount|
+----+----------+
|  12|       421|
|  13|       268|
|  16|       160|
|  15|       189|
|   9|       151|
|  17|       647|
|   8|        46|
|  10|       121|
|  11|       370|
|  14|       735|
+----+----------+



In [10]:
df.groupBy("Country", "StockCode").agg(count("*").alias("ProductBoughtFrequency")).show()

+--------------+---------+----------------------+
|       Country|StockCode|ProductBoughtFrequency|
+--------------+---------+----------------------+
|United Kingdom|    21125|                     1|
|United Kingdom|    21794|                     1|
|United Kingdom|    21662|                     2|
|          EIRE|   85136C|                     1|
|United Kingdom|   84032A|                     3|
|United Kingdom|   84086C|                     2|
|United Kingdom|   90214H|                     2|
|United Kingdom|    22753|                     1|
|United Kingdom|    21557|                     2|
|United Kingdom|    21743|                     4|
|United Kingdom|    22488|                     1|
|United Kingdom|    22956|                     3|
|        Norway|    22531|                     1|
|United Kingdom|    22728|                     3|
|United Kingdom|    82551|                     2|
|United Kingdom|    22581|                     2|
|United Kingdom|    21671|                     1|


In [11]:
df.where(expr("Country LIKE 'United %'")).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S